In [60]:
%matplotlib inline
from utils import read_fits_to_pandas

In [56]:
from astropy.table import Table
apogee = Table.read('/data/jls/GaiaDR2/spectro/APOGEE_distances_withPRIOR_MASTER.hdf5')
apogee_inputs = pd.read_hdf('/data/jls/GaiaDR2/spectro/APOGEE_input_MASTER.hdf5')
print 'Processed:',np.count_nonzero(apogee['flag']==0.), 'out of', len(apogee)
rave = Table.read('/data/jls/GaiaDR2/spectro/RAVE_DR5_distances_withPRIOR_MASTER.hdf5')
rave_inputs = pd.read_hdf('/data/jls/GaiaDR2/spectro/RAVE_input_MASTER.hdf5')
print 'Processed:',np.count_nonzero(rave['flag']==0.), 'out of', len(rave)
ges = Table.read('/data/jls/GaiaDR2/spectro/GES_DR3_distances_withPRIOR_MASTER.hdf5')
ges_inputs = pd.read_hdf('/data/jls/GaiaDR2/spectro/GES_input_MASTER.hdf5')
print 'Processed:',np.count_nonzero(ges['flag']==0.), 'out of', len(ges)
galah = Table.read('/data/jls/GaiaDR2/spectro/GALAH_distances_withPRIOR_MASTER.hdf5')
galah_inputs = pd.read_hdf('/data/jls/GaiaDR2/spectro/GALAH_input_MASTER.hdf5')
print 'Processed:',np.count_nonzero(galah['flag']==0.), 'out of', len(galah)
lamost = Table.read('/data/jls/GaiaDR2/spectro/LAMOST_distances_withPRIOR_MASTER.hdf5')
lamost_inputs = pd.read_hdf('/data/jls/GaiaDR2/spectro/LAMOST_input_MASTER.hdf5')
print 'Processed:',np.count_nonzero(lamost['flag']==0.), 'out of', len(lamost)
raveON = Table.read('/data/jls/GaiaDR2/spectro/RAVE_Cannon_distances_withPRIOR_MASTER.hdf5')
raveON_inputs = pd.read_hdf('/data/jls/GaiaDR2/spectro/RAVEON_input_MASTER.hdf5')
print 'Processed:',np.count_nonzero(raveON['flag']==0.), 'out of', len(raveON)

from astropy.table import Table, vstack, join

def flag_errors(data, covar_correction=True):
    for col in ['dist_err', 'par_err', 'dm_err',
                'log10_teff_err', 'logg_err', 'log10_av_err',
                'log10_age_err',
                'mass_err',
                'Z_err', 'dm_log10age_corr', 'log10age_Z_corr', 'dm_Z_corr']:
        data['flag'][(data['flag'] == 0) & (data[col] != data[col])] = 6
        data['flag'][(data['flag'] == 0) & (data[col] == 0.)] = 6
    data['flag'][(data['log10_age'] < -1)&(data['logg']>3.5)&(data['log10_teff']<3.9)] = 7
    for col in ['dm_log10age_corr', 'log10age_Z_corr', 'dm_Z_corr']:
        data['flag'][(data['flag'] == 0) & (np.abs(data[col]) > 1.)] = 6
    return data

def fillin(r, rfill, fld):
    r['indx'] = np.arange(len(r))
    rfill['indx'] = np.arange(len(rfill))
    r2 = join(r[[fld, 'indx']],
              rfill[[fld, 'indx']],
              keys=fld, join_type='left')
    r[r2['indx_1'][~r2['indx_2'].mask]] = \
        rfill[r2['indx_2'][~r2['indx_2'].mask]]
    r.remove_column('indx')
    return r

segue = Table.read('/data/jls/GaiaDR2/spectro/SEGUE_distances_withPRIOR_MASTER.hdf5')
rfill = Table.read(
       '/data/jls/GaiaDR2/spectro/SEGUE_distances_withPRIOR_parallax_fillin.hdf5')
rfill = flag_errors(rfill)
segue = fillin(segue, rfill, 'specobjid')
segue_inputs = pd.read_hdf('/data/jls/GaiaDR2/spectro/SEGUE_input_MASTER.hdf5')
print 'Processed:',np.count_nonzero(segue['flag']==0.), 'out of', len(segue)

Processed: 203417 out of 258475
Processed: 415200 out of 457555
Processed: 10882 out of 25332
Processed: 260233 out of 342682
Processed: 2861310 out of 3177995
Processed: 376316 out of 457555
Processed: 182132 out of 187152


In [57]:
la = pd.read_hdf('/data/jls/GaiaDR2/spectro/lamost_cannon/LAMOST_results.hdf5')
lamost_inputs = lamost_inputs.merge(la,on='obsid',how='left')

In [58]:
len(lamost_inputs), len(la)

(3177995, 728907)

In [61]:
ges = read_fits_to_pandas('/data/jls/ges/GES_iDR4_WG15_Recommended_v2.fits')
ges_inputs = ges_inputs.merge(ges,on='CNAME',how='left')

In [71]:
import sqlutil
from login import wsdbpassword
def grab_all_raveon():
    data = sqlutil.get("""select * from rave_on_1_0.main as r""",
                       host='cappc127', user='jason_sanders',
                       password=wsdbpassword,
                       preamb='set enable_seqscan to off; ' +
                              'set enable_mergejoin to off; ' +
                              'set enable_hashjoin to off;', asDict=True,
                       strLength=30)
    df = pd.DataFrame(columns=data.keys())
    for k in data.keys():
        df[k] = data[k]
    return df

In [127]:
rr = grab_all_raveon()

In [128]:
raveON_inputs = pd.merge(raveON_inputs,rr,on='rave_obs_id',how='left')
# plt.plot(rr['fe_h'],rr['mg_h']-rr['fe_h'],'.',ms=1)

In [146]:
lamost_inputs['alpha_fe']=lamost_inputs['ALPHA_M']
lamost_inputs['e_alpha_fe']=lamost_inputs['ALPHA_M_ERR_SNR']
raveON_inputs['alpha_fe']=raveON_inputs['mg_h_y']-raveON_inputs['fe_h_y']
raveON_inputs['e_alpha_fe']=np.sqrt(raveON_inputs['e_mg_h_y']**2+raveON_inputs['e_fe_h_y']**2)
raveON_inputs['fe_h']=raveON_inputs['fe_h_x']
raveON_inputs['e_fe_h']=raveON_inputs['e_fe_h_x']
apogee_inputs['alpha_fe']=apogee_inputs['ALPHA_M']
apogee_inputs['e_alpha_fe']=apogee_inputs['ALPHA_M_ERR']
galah_inputs['alpha_fe']=galah_inputs['alpha_fe']
galah_inputs['e_alpha_fe']=galah_inputs['e_alpha_fe']
ges_inputs['alpha_fe']=ges_inputs['ALPHA_FE']
ges_inputs['e_alpha_fe']=ges_inputs['E_ALPHA_FE']

segue_inputs['alpha_fe']=np.nan
segue_inputs['e_alpha_fe']=np.nan

segue_inputs['J']=np.nan
segue_inputs['H']=np.nan
segue_inputs['K']=np.nan
segue_inputs['eJ']=np.nan
segue_inputs['eH']=np.nan
segue_inputs['eK']=np.nan


rave_inputs['alpha_fe']=np.nan
rave_inputs['e_alpha_fe']=np.nan

In [144]:
raveON_inputs['raveid']=raveON_inputs['raveid_x']

In [65]:
joint_table = Table.read('/home/jls/public_html/data/gaia_spectro.hdf5')

In [147]:
flds = ['APOGEE_ID','sobject_id','CNAME','obsid','raveid','raveid','specobjid']
g = joint_table[['survey','APOGEE_ID','sobject_id','CNAME','obsid','raveid','specobjid','source_id']].copy()
g['indx']=np.arange(len(g))
cols = ['fe_h','e_fe_h','alpha_fe','e_alpha_fe',
           'J','H','K','G','GBP','GRP',
           'eJ','eH','eK','eG','eGBP','eGRP']
for cc in cols:
    g[cc]=np.nan

for dd,fld,s in zip([apogee_inputs,galah_inputs,ges_inputs,lamost_inputs,rave_inputs,raveON_inputs,segue_inputs],
                    flds,
                    ['APOGEE','GALAH','GES','LAMOST','RAVEDR5','RAVEON','SEGUE']):
    print s
    dd['survey']=s
    dd['ind']=1
    for cc in cols:
        dd[cc+'_T']=dd[cc]
    cols_use = [fld,'survey','ind']+[c+'_T' for c in cols]
    g = join(g,Table.from_pandas(dd[cols_use]),
                 keys=[fld,'survey'],join_type='left')
    for cc in cols:
        g[cc][g['ind']==1]=g[cc+'_T'][g['ind']==1]
    del g['ind']
    for cc in cols:
        del g[cc+'_T']
g.sort(keys='indx')
del g['indx']

APOGEE
GALAH
GES
LAMOST
RAVEDR5
RAVEON
SEGUE


In [153]:
g.write('/data/jls/GaiaDR2/spectro/input_photometry_and_spectroscopy.hdf5',
        format='hdf5',path='input_photometry_and_spectroscopy')

In [154]:
np.count_nonzero(g['alpha_fe']>-10.)

1429719